## Create DataFrame

In [0]:
dbutils.library.restartPython() # Removes Python state, but some libraries might not work without calling this command.dbutils.restartPython()

#### Load libraries

In [0]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import IntegerType, DateType, StringType, StructType, StructField

#### Create Spark session

In [0]:
spark = SparkSession.builder.appName('Create dataframe').getOrCreate()

#### Prepare data

In [0]:
columns = ['Programming Language', 'Ratings']
data = [('C', 12.54),
('Python', 11.84),
('Java', 11.54),
('C++', 7.36),
('C#', 4.33),
('Visual Basic', 4.01),
('JavaScript', 2.33),
('PHP', 2.21),
('Assembly language', 2.05),
('SQL', 1.88)]

#### Create DataFrame from RDD

In [0]:
rdd = spark.sparkContext.parallelize(data)

#### Using toDF() function

In [0]:
# RDD doesn’t have columns, the DataFrame is created with default column names
dfFromRDD = rdd.toDF()
dfFromRDD.printSchema()

root
-- _1: string (nullable = true)
-- _2: double (nullable = true)

In [0]:
# With columns
dfFromRDDc = rdd.toDF(columns)
dfFromRDDc.printSchema()

root
-- Programming Language: string (nullable = true)
-- Ratings: double (nullable = true)

#### Using createDataFrame() from SparkSession

In [0]:
dfFromRDDs = spark.createDataFrame(rdd).toDF(*columns)
dfFromRDDs.printSchema()

root
-- Programming Language: string (nullable = true)
-- Ratings: double (nullable = true)

#### Create DataFrame from List Collection

In [0]:
# Using createDataFrame() from SparkSession
dfFromDataList = spark.createDataFrame(data).toDF(*columns)
dfFromDataList.printSchema()

root
-- Programming Language: string (nullable = true)
-- Ratings: double (nullable = true)

In [0]:
# Using createDataFrame() with the Row type

rowData = map(lambda x: Row(*x), data) 
dfFromDataRow = spark.createDataFrame(rowData,columns)
dfFromDataRow.printSchema()

root
-- Programming Language: string (nullable = true)
-- Ratings: double (nullable = true)

#### Create DataFrame with schema

In [0]:
# Populate some data
data2 = [
  ('John', '', 'Smith', '36636', 'M', 2500),
  ('Jane', '', 'Doe', '42114', 'F', 500),
  ('Richard', 'Laurence', 'Marquette', 97086, 'M', 1500),
  ('Israel', '', 'Israeli', '', 'M', 3000),
  ('Edward', 'III', '', 'SL4', 'M', 5000)
]
 
schema = StructType([
  StructField("firstname",StringType(),True),
  StructField("middlename",StringType(),True),
  StructField("lastname",StringType(),True),
  StructField("id", StringType(), True),
  StructField("gender", StringType(), True),
  StructField("salary", IntegerType(), True)
])
 
df = spark.createDataFrame(data=data2, schema=schema)
df.printSchema()
df.show(truncate=False)

root
-- firstname: string (nullable = true)
-- middlename: string (nullable = true)
-- lastname: string (nullable = true)
-- id: string (nullable = true)
-- gender: string (nullable = true)
-- salary: integer (nullable = true)

+---------+----------+---------+-----+------+------+
firstname|middlename|lastname |id |gender|salary|
+---------+----------+---------+-----+------+------+
John | |Smith |36636|M |2500 |
Jane | |Doe |42114|F |500 |
Richard |Laurence |Marquette|97086|M |1500 |
Israel | |Israeli | |M |3000 |
Edward |III | |SL4 |M |5000 |
+---------+----------+---------+-----+------+------+

#### Create DataFrame from Data sources

In real-time mostly you create DataFrame from data source files like CSV, Text, JSON, XML e.t.c.

PySpark by default supports many data formats out of the box without importing any libraries and to create DataFrame you need to use the appropriate method available in **DataFrameReader** class.

##### Creating DataFrame from CSV

Use `csv()` method of the `DataFrameReader` object to create a DataFrame from CSV file. you can also provide options like what delimiter to use, whether you have quoted data, date formats, infer schema, and many more.

    df2 = spark.read.csv("/src/resources/file.csv")

##### Creating from text (TXT) file

Similarly you can also create a DataFrame by reading a from Text file, use `text()` method of the `DataFrameReader` to do so.

    df2 = spark.read.text("/src/resources/file.txt")

##### Creating from JSON file

PySpark is also used to process semi-structured data files like JSON format. you can use `json()` method of the `DataFrameReader` to read JSON file into DataFrame.

    df2 = spark.read.json("/src/resources/file.json")

In [0]:
v_source_file = "/FileStore/tables/ufo.csv"

df_csv = (
  spark
  .read
  .format('csv')
  .option('header','true')
  .option('inferSchema', 'false')  
  .option('delimiter',',')                                
  .option('quote', '\"')
  .load(v_source_file)
)

df_csv.show(5)

+------------+-----------+------------+----------+-----+---------+--------+
 LATITUDE| LONGITUDE| CITY|DATE_OCCUR|STATE|UFO_SHAPE|DURATION|
+------------+-----------+------------+----------+-----+---------+--------+
-95.78950112|36.05084018|Broken Arrow|11/22/2008| OK| Changing| 0|
-118.2453206|34.05349094| Los Angeles|04/08/2006| CA| Changing| 0|
-106.6486413|35.08418003| Albuquerque|05/26/2007| NM| Circle| 0|
-71.53660023|43.20725072| Concord|06/15/2001| NH| Circle| 0|
-76.51119034|43.45646037| Oswego|11/10/2005| NY| Circle| 0|
+------------+-----------+------------+----------+-----+---------+--------+
only showing top 5 rows

#### The end of the notebook